In [1]:
import urllib.error
from urllib.request import urlopen, Request

def url_download(url, user_agent="wswp" ,num_retries = 0):
    print("Downloading url :" + str(url))
    headers = {'User-Agent': 'Mozilla/5.0'}
    request = Request(url, headers=headers)
    try:
        html = urlopen(request).read()
        html = html.decode('utf-8')
    except urllib.error.HTTPError as e:
        print('HTTPError: {}'.format(e.code))
        if (e.code >= 500 and e.code < 600 and num_retries>0):
            html = url_download(url,num_retries-1)
        else:
            html = None
    except urllib.error.URLError as e:
        print('URLError: {}'.format(e.reason))
        html = None
    except SocketError as e:
        print('SocketError: {}'.format(e))
        html = None
    except:
        print(e)
    return html

In [2]:
html = url_download("https://www.urbanclap.com/mumbai-ac-service-repair")

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html,'html.parser')

In [4]:
a = soup.find('div',attrs={'id':'firstQuestion'})

In [5]:
b = a.findAll('button')

In [6]:
for element in b:
    print(element.get_text())

AC Servicing
AC not cooling/ Repair
AC Installation/ Un-Installation


## Scraping the complete data

In [4]:
location_info=[{'name': 'Ahmedabad', 'value': 'ahmedabad'}, {'name': 'Bangalore', 'value': 'bangalore'}, {'name': 'Chennai', 'value': 'chennai'}, {'name': 'Delhi NCR', 'value': 'delhi'}, {'name': 'Chandigarh Tricity', 'value': 'chandigarh'}, {'name': 'Jaipur', 'value': 'jaipur'}, {'name': 'Dubai', 'value': 'dubai'}, {'name': 'Abu Dhabi', 'value': 'abudhabi'}, {'name': 'Hyderabad', 'value': 'hyderabad'}, {'name': 'Kolkata', 'value': 'kolkata'}, {'name': 'Mumbai', 'value': 'mumbai'}, {'name': 'Pune', 'value': 'pune'}]
allData = []

In [64]:
import requests

def extract_info(data):
    question = []
    for q in data:
        try:
            details = {'title':q['title'],'meta_data':q['meta_data']['data']}
        except:
            details = {}
        question.append(details)
    return question

def get_details_of_service_data(key_name,city_name):
    url = "https://www.urbanclap.com/api/v1/categoryquestions/get"

    payload = "device_id=ucu7db60-bd56e9f94c-aa23-9d9a-a7ab-f6f8813d23-1564129250861&city_key={0}&category_key_name={1}&app_version=4.12.12&device=web&is_auto=true".format(city_name,key_name)
    headers = {
    'Content-Type': "application/x-www-form-urlencoded",
    'User-Agent': "PostmanRuntime/7.15.2",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Postman-Token': "fdf674c1-6f13-46f0-a186-97552cdd5df5,9feb8c82-71eb-4f58-b4bf-2c2b87c7c44f",
    'Host': "www.urbanclap.com",
    'Cookie': "__cfduid=da1f11335a25c1b20e81c8e7ac6d1b1391564132893",
    'Accept-Encoding': "gzip, deflate",
    'Content-Length': "176",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }

    response = requests.request("POST", url, data=payload, headers=headers)
    
    try:
        r=response.json()['success']['data']['questions']
        r = extract_info(r)
    except:
        r=-1
        print(city_name,key_name)
    return r

def get_service_data(d):
    city_key = d["city_key"]
    service_details = {"key_name": d['key_name'], "display_name": d['display_name'],"city_key":city_key}
    try:
        service_details['url'] = d['url']
    except:
        service_details['url'] = None
        
    service_details["serivce_detail_info"] = get_details_of_service_data(d['key_name'],city_key)
    
    return service_details

def get_location_services(city_name):
    url = "https://www.urbanclap.com/api/v1/customercategories/search/category/city_{0}_v2/a".format(city_name)
    r = requests.get(url = url)
    data = r.json()
    data = data['success']['data']
    services = [get_service_data(d) for d in data]
    return services

In [71]:
full_data = []
for location in location_info:
    city_name = location['value']
    details = {"city_name" : city_name}
    services = get_location_services(city_name)
    details['services']=services
    full_data.append(details)

city_mumbai_v2 uc_mumo_temp_ondemand


In [72]:
import json
with open('UrbanClap_data.json', 'w', encoding='utf-8') as f:
    json.dump(full_data, f, ensure_ascii=False, indent=4)